# Validate PATs

In [49]:
import os

In [50]:
# load packages 
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
import requests as r
import string 
import json
import base64
import urllib.request
import itertools 
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from github import Github, RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [107]:
#os.environ['db_user'] = ''
#os.environ['db_pwd'] = ''

# connect to the database, download data, limit to repos with at least 20,000 commits?
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))


#PATs access token, saved as a dataframe
github_pats = '''SELECT * FROM gh_2007_2020.pats'''
github_pats = pd.read_sql_query(github_pats, con=connection)
github_usernames = github_pats["login"]

#PATs access token, saved as a list
access_tokens = github_pats["token"]
#print(access_tokens.head())

#number of tokens available for use, a numeric value
num_token = '''SELECT COUNT(*) FROM gh_2007_2020.pats'''
num_token = pd.read_sql_query(num_token, con=connection)
num_token=num_token.iloc[0]['count']

connection.close()

In [96]:
slugs = ["moderndive/ModernDive_book", "DSPG-Young-Scholars-Program/dspg21oss", "DSPG-Young-Scholars-Program/dspg21RnD"]

In [97]:
#print(github_username)
#print(access_tokens)

In [98]:
print(len(access_tokens))
pats = access_tokens[10:len(access_tokens)]
#print(pats)

36


In [100]:

def validate_pat_for_repo_stats(slug):
    i=0
    df_repo = pd.DataFrame()
    while i <len(access_tokens):
        while True:
            try:
                access_token = access_tokens[i]
                g = Github(access_token)
                print("Using access token #",i,"--", access_token)
                repo = g.get_repo(slug)
                df_repo = df_repo.append({
                    "access_token": access_token,
                    "token_index": i,
                    "slug": slug,
                    'stars': repo.stargazers_count,
                    'watchers': repo.subscribers_count,
                    'forks': repo.forks_count,
                    'topics': repo.get_topics(),
                }, ignore_index = True)
                i+=1
            except BadCredentialsException as e:
                print(e.status)
                print('Bad credentials exception using access token #',i,"--", access_token)
                i+=1
                break
    return df_repo

In [102]:
slug = "moderndive/ModernDive_book"
df_repo = validate_pat_for_repo_stats(slug)

Using access token # 0 -- 51f7b11ebd070ebb473d65a5a8f90cdf1c71a3a4
401
Bad credentials exception using access token # 0 -- 51f7b11ebd070ebb473d65a5a8f90cdf1c71a3a4
Using access token # 1 -- c038b80105bebef08d9115cd96243f7a33967352
401
Bad credentials exception using access token # 1 -- c038b80105bebef08d9115cd96243f7a33967352
Using access token # 2 -- 89450cbf81387c2bc3fe9277dce13031ef306f26
401
Bad credentials exception using access token # 2 -- 89450cbf81387c2bc3fe9277dce13031ef306f26
Using access token # 3 -- d221261a4e833f66123fd58f1ffd1b728cd580df
401
Bad credentials exception using access token # 3 -- d221261a4e833f66123fd58f1ffd1b728cd580df
Using access token # 4 -- a864fdd3faa50a01e355a596b6c6fbfc137df3f4
401
Bad credentials exception using access token # 4 -- a864fdd3faa50a01e355a596b6c6fbfc137df3f4
Using access token # 5 -- 1d5f851579881c92aa06e1d7366df3a23efcd5a0
Using access token # 6 -- 0b1854c595b2d6ddc86e7dd8653cc2e563ec3b13
Using access token # 7 -- 5681dcb3e17b5520742f

In [91]:
df_repo

,access_token,forks,slug,stars,token_index,topics,watchers
0,1d5f851579881c92aa06e1d7366df3a23efcd5a0,294.0,moderndive/ModernDive_book,548.0,5.0,"[moderndive, data-science, tidyverse, statisti...",28.0
1,0b1854c595b2d6ddc86e7dd8653cc2e563ec3b13,294.0,moderndive/ModernDive_book,548.0,6.0,"[moderndive, data-science, tidyverse, statisti...",28.0
2,5681dcb3e17b5520742f6cca324c7c661637c1e9,294.0,moderndive/ModernDive_book,548.0,7.0,"[moderndive, data-science, tidyverse, statisti...",28.0
3,a70abe9256cde92b51077eb5915b468a22389170,294.0,moderndive/ModernDive_book,548.0,8.0,"[moderndive, data-science, tidyverse, statisti...",28.0
4,1c225a39a99830c722ef2567b3196a61ab245dbe,294.0,moderndive/ModernDive_book,548.0,9.0,"[moderndive, data-science, tidyverse, statisti...",28.0
5,4ff1d48a6cbaba793449ade89d920f7d64f8ecee,294.0,moderndive/ModernDive_book,548.0,10.0,"[moderndive, data-science, tidyverse, statisti...",28.0
6,cebc7eaae494a84870c95c899ec616dd0270f12b,294.0,moderndive/ModernDive_book,548.0,11.0,"[moderndive, data-science, tidyverse, statisti...",28.0
7,c3d7e8d27488c15f24eb45bd06ff35abc74f9239,294.0,moderndive/ModernDive_book,548.0,12.0,"[moderndive, data-science, tidyverse, statisti...",28.0
8,ac4b7d6f034f69ef306cd1e7b04b1291036c3afb,294.0,moderndive/ModernDive_book,548.0,13.0,"[moderndive, data-science, tidyverse, statisti...",28.0
9,bb1d1387b91d4b70865a6f8043ae46ce1de828ff,294.0,moderndive/ModernDive_book,548.0,14.0,"[moderndive, data-science, tidyverse, statisti...",28.0


In [135]:
def validate_pat_for_readme(slug):
    i=0
    df_repo = pd.DataFrame()
    pat_invalid = list()
    while i <len(access_tokens):
        while True:
            try:
                url = f'https://api.github.com/repos/{slug}/readme'
                access_token = access_tokens[i]
                github_username = github_usernames[i]
                
                print("Using access token #",i,"--", github_username, "--", access_token)

                response = r.get(url, auth=(github_username, access_token))                
                html_content = response.content
                soup = BeautifulSoup(html_content, 'html.parser')
                site_json=json.loads(soup.text)
                readme_link = site_json['download_url']

                readme_response = r.get(readme_link, auth=(github_username, access_token))
                readme_response_code = readme_response.status_code

                # pull the content out of the readme 
                readme_content = readme_response.content
                readme_soup = BeautifulSoup(readme_content, 'html.parser')
                readme_string = str(readme_soup)
                i+=1
            except KeyError:
                print("Key error using access token #" ,i, ":", access_token, flush=True)
                pat_invalid.append(access_token)
                i+= 1
                break
    return i, access_token, readme_string, pat_invalid

In [136]:
readme_df = validate_pat_for_readme(slug)

Using access token # 0 -- akindlon977 -- 51f7b11ebd070ebb473d65a5a8f90cdf1c71a3a4
Key error using access token # 0 : 51f7b11ebd070ebb473d65a5a8f90cdf1c71a3a4
Using access token # 1 -- Azraab -- c038b80105bebef08d9115cd96243f7a33967352
Key error using access token # 1 : c038b80105bebef08d9115cd96243f7a33967352
Using access token # 2 -- Behnaz-m -- 89450cbf81387c2bc3fe9277dce13031ef306f26
Key error using access token # 2 : 89450cbf81387c2bc3fe9277dce13031ef306f26
Using access token # 3 -- brandonleekramer -- d221261a4e833f66123fd58f1ffd1b728cd580df
Key error using access token # 3 : d221261a4e833f66123fd58f1ffd1b728cd580df
Using access token # 4 -- CalvinIsch -- a864fdd3faa50a01e355a596b6c6fbfc137df3f4
Key error using access token # 4 : a864fdd3faa50a01e355a596b6c6fbfc137df3f4
Using access token # 5 -- christawang -- 1d5f851579881c92aa06e1d7366df3a23efcd5a0
Using access token # 6 -- cohense -- 0b1854c595b2d6ddc86e7dd8653cc2e563ec3b13
Using access token # 7 -- dads2busy -- 5681dcb3e17b552

In [139]:
pat_invalid

NameError: name 'pat_invalid' is not defined